In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>", fig.align = "center"
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)


packageVersion("Voyager")

In [ ]:
# Install anndata for cellatlas examples
# install.packages("reticulate")
reticulate::install_miniconda(force = TRUE)
install.packages("anndata")

# Introduction
The data in this vignette is shipped with the `cellatlas` repository. The count matrix and metadata are provided in the `cellatlas/examples` folder as an [`AnnData`](https://anndata-tutorials.readthedocs.io/en/latest/getting-started.html) object. We will begin by loading the object and converting it to a `SpatialFeatureExperiment` object. 

In [ ]:
library(zellkonverter)
library(stringr)
library(Matrix)
library(SpatialExperiment)
library(SpatialFeatureExperiment)
library(scater)
library(scuttle)
library(Voyager)

In [ ]:
download.file("https://github.com/cellatlas/cellatlas/raw/main/examples/rna-splitseq/cellatlas_out/adata.h5ad.gz", 
              "adata.h5ad.gz", quiet = TRUE)
system("gunzip -f adata.h5ad.gz")
sce <- readH5AD("adata.h5ad")
assayNames(sce) <- "counts"

In [ ]:
is_mito <- str_detect(rowData(sce)$gene_name, regex("^mt-", ignore_case=TRUE))
sum(is_mito)

In [ ]:
sce <- addPerCellQCMetrics(sce, subsets = list(mito = is_mito))
names(colData(sce))

In [ ]:
plotColData(sce, "sum") +
    plotColData(sce, "detected") +
    plotColData(sce, "subsets_mito_percent")

In [ ]:
plotColData(sce, x = "sum", y = "detected", bins = 100) +
    scale_fill_distiller(palette = "Blues", direction = 1)

In [ ]:
plotColData(sce, x = "sum", y = "subsets_mito_detected", bins = 100) +
    scale_fill_distiller(palette = "Blues", direction = 1)

In [ ]:
sce <- sce[, which(sce$subsets_mito_percent < 20)]
sce <- sce[rowSums(counts(sce)) > 0,]

sce

In [ ]:
sessionInfo()